In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:


import pandas as pd
%cd /content/gdrive/MyDrive/0. 최종 프로젝트/데이터 탐색/
import os

# train, valid, test file 경로 지정
path = '/content/gdrive/MyDrive/0. 최종 프로젝트/데이터 탐색/'



# train data 로딩
df = pd.read_csv("physionet2012_train.csv")

del df['Unnamed: 0']
df.head()

# valid data 로딩
valid_df = pd.read_csv('physionet2012_valid.csv')
del valid_df['Unnamed: 0']
valid_df.head()

# tess data 로딩
test_df = pd.read_csv('physionet2012_test.csv')
del test_df['Unnamed: 0']
test_df.head()

/content/gdrive/MyDrive/0. 최종 프로젝트/데이터 탐색


,pid,rec_time,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,...,pH,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS
0,135089,0:00:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7,6
1,135089,0:44:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7,6
2,135089,2:09:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7,6
3,135089,2:10:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7,6
4,135089,2:15:00,90.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,116.0,58.0,77.0,1,0,-7,6


In [ ]:
import numpy as np
# train 데이터 생존 유무 라벨링
labels = np.zeros(len(df))
labels[df[(2 <= df.survival) & (df.survival <= df.LOS)].index.to_list()] = 1
labels[df[df.survival > df.LOS].index.to_list()] = 0
labels[df[df.survival == -1].index.to_list()] = 0
df['labels'] = labels

cnt = df[['pid', 'labels']].drop_duplicates(['pid'])['labels'].value_counts()
print(f'num of survival: {cnt.iloc[0]} \n num of death: {cnt.iloc[1]}')

# valid data 생존 유무 라벨링
valid_labels = np.zeros(len(valid_df))
valid_labels[valid_df[(2 <= valid_df.survival) & (valid_df.survival <= valid_df.LOS)].index.to_list()] = 1
valid_labels[valid_df[valid_df.survival > valid_df.LOS].index.to_list()] = 0
valid_labels[valid_df[valid_df.survival == -1].index.to_list()] = 0
valid_df['labels'] = valid_labels

cnt = valid_df[['pid', 'labels']].drop_duplicates(['pid'])['labels'].value_counts()
print(f'num of survival: {cnt.iloc[0]} \n num of death: {cnt.iloc[1]}')

# test data 생존 유무 라벨링
test_labels = np.zeros(len(test_df))
test_labels[test_df[(2 <= test_df.survival) & (test_df.survival <= test_df.LOS)].index.to_list()] = 1
test_labels[test_df[test_df.survival > test_df.LOS].index.to_list()] = 0
test_labels[test_df[test_df.survival == -1].index.to_list()] = 0
test_df['labels'] = test_labels

cnt = test_df[['pid', 'labels']].drop_duplicates(['pid'])['labels'].value_counts()
print(f'num of survival: {cnt.iloc[0]} \n num of death: {cnt.iloc[1]}')

num of survival: 2813 
 num of death: 427
num of survival: 311 
 num of death: 49
num of survival: 348 
 num of death: 52


In [ ]:
# train 데이터 결측치 채우기
df2 = df.copy()
pids = df2.pid.unique()
df_list = []
for pidd in pids:
    # 환자별 컬럼별로 만약 np.nan이 존재한다면, 이전 데이터의 값을 가지고 채운다. method='ffill'
    new_col = df2[df2.pid == pidd].fillna(method='ffill', axis=0)
    # 나머지 결측치들을 다른 환자의 데이터의 median으로 채우기 위해, median 계산이 어려운 'rec_time'을 따로 분리한다.    
    rec_time_col = pd.DataFrame(new_col['rec_time'], columns=['rec_time'])
    new_col.drop(columns=['rec_time'], inplace=True)
    new_col = new_col.fillna(df2.median())
    new_df = pd.concat([rec_time_col, new_col], axis=1)
    df_list.append(new_df)
df3 = pd.concat(df_list)

# valid 데이터 결측치 채우기
valid_df2 = valid_df.copy()
pids = valid_df2.pid.unique()
valid_df_list = []
for pidd in pids:
    # 환자별 컬럼별로 만약 np.nan이 존재한다면, 이전 데이터의 값을 가지고 채운다. method='ffill'
    new_col = valid_df2[valid_df2.pid == pidd].fillna(method='ffill', axis=0)
    # 나머지 결측치들을 다른 환자의 데이터의 median으로 채우기 위해, median 계산이 어려운 'rec_time'을 따로 분리한다.    
    rec_time_col = pd.DataFrame(new_col['rec_time'], columns=['rec_time'])
    new_col.drop(columns=['rec_time'], inplace=True)
    new_col = new_col.fillna(valid_df2.median())
    new_df = pd.concat([rec_time_col, new_col], axis=1)
    valid_df_list.append(new_df)
valid_df3 = pd.concat(valid_df_list)

# test 데이터 결측치 채우기
test_df2 = test_df.copy()
pids = test_df2.pid.unique()
test_df_list = []
for pidd in pids:
    # 환자별 컬럼별로 만약 np.nan이 존재한다면, 이전 데이터의 값을 가지고 채운다. method='ffill'
    new_col = test_df2[test_df2.pid == pidd].fillna(method='ffill', axis=0)
    # 나머지 결측치들을 다른 환자의 데이터의 median으로 채우기 위해, median 계산이 어려운 'rec_time'을 따로 분리한다.    
    rec_time_col = pd.DataFrame(new_col['rec_time'], columns=['rec_time'])
    new_col.drop(columns=['rec_time'], inplace=True)
    new_col = new_col.fillna(test_df2.median())
    new_df = pd.concat([rec_time_col, new_col], axis=1)
    test_df_list.append(new_df)
test_df3 = pd.concat(test_df_list)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  # This is

In [ ]:
# 데이터 개수와 컬럼별 null 개수 확인
print(len(df3))
pd.DataFrame(df3.isnull().sum())
pd.DataFrame(df3.Albumin.value_counts())

print(len(valid_df3))
pd.DataFrame(valid_df3.isnull().sum())

print(len(test_df3))
pd.DataFrame(test_df3.isnull().sum())

df3.to_csv(path+'df3.csv', index=False)
valid_df3.to_csv(path+'valid_df3.csv', index=False)
test_df3.to_csv(path+'test_df3.csv', index=False)

242613
27097
29554


In [ ]:
df3 = pd.read_csv(path+'df3.csv')
valid_df3 = pd.read_csv(path+'valid_df3.csv')
test_df3 = pd.read_csv(path+'test_df3.csv')

In [ ]:
# 데이터가 단 하나도 없는 column 'Tropl', 'TropT' 제외
# 'rec_time'은 이후 padding에서 np.zeros에 string 데이터가 들어갈 수 없는 문제를 해결하기 위해 미리 제거
df4 = df3.drop(columns=['Tropl', 'TropT', 'rec_time'])
pd.DataFrame(df4.isnull().sum())

valid_df4 = valid_df3.drop(columns=['Tropl', 'TropT', 'rec_time'])
pd.DataFrame(valid_df4.isnull().sum())

test_df4 = test_df3.drop(columns=['Tropl', 'TropT', 'rec_time'])
pd.DataFrame(test_df4.isnull().sum())

,0
pid,0
age,0
Gender,0
ICUType,0
ALP,0
ALT,0
AST,0
Albumin,0
BUN,0
Bilirubin,0


In [ ]:
df4.head()

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,...,MechVent,survival,SBP,DBP,MBP,male,female,surv_los,LOS,labels
0,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,1.0,-1,116.0,58.0,77.0,1,0,-8,7,0.0
1,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,1.0,-1,116.0,58.0,77.0,1,0,-8,7,0.0
2,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,1.0,-1,0.0,0.0,224.0,1,0,-8,7,0.0
3,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,1.0,-1,0.0,0.0,135.0,1,0,-8,7,0.0
4,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,1.0,-1,217.0,108.0,133.0,1,0,-8,7,0.0


In [ ]:
df4=df4.assign(Score="")

In [ ]:
df4.fillna(0)

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,...,survival,SBP,DBP,MBP,male,female,surv_los,LOS,labels,Score
0,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,116.0,58.0,77.0,1,0,-8,7,0.0,
1,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,116.0,58.0,77.0,1,0,-8,7,0.0,
2,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,0.0,0.0,224.0,1,0,-8,7,0.0,
3,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,0.0,0.0,135.0,1,0,-8,7,0.0,
4,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,217.0,108.0,133.0,1,0,-8,7,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,17,114.0,65.0,84.0,1,0,-1,18,1.0,
242609,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,17,111.0,61.0,80.0,1,0,-1,18,1.0,
242610,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,17,109.0,62.0,80.0,1,0,-1,18,1.0,
242611,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,17,111.0,63.0,80.0,1,0,-1,18,1.0,


In [ ]:
df4.head()

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,...,survival,SBP,DBP,MBP,male,female,surv_los,LOS,labels,Score
0,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,116.0,58.0,77.0,1,0,-8,7,0.0,
1,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,116.0,58.0,77.0,1,0,-8,7,0.0,
2,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,0.0,0.0,224.0,1,0,-8,7,0.0,
3,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,0.0,0.0,135.0,1,0,-8,7,0.0,
4,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,-1,217.0,108.0,133.0,1,0,-8,7,0.0,


In [ ]:

df4['GCS'].value_counts()

15.0    88949
3.0     27560
10.0    25156
13.0    19955
11.0    15860
14.0    13696
8.0     12953
7.0     12775
9.0     11310
6.0      9046
12.0     2104
4.0      1743
5.0      1506
Name: GCS, dtype: int64

In [ ]:
df4=df4.assign(AP_Tem="")
df4=df4.assign(AP_MBP="")
df4=df4.assign(AP_HR="")
df4=df4.assign(AP_RR="")
df4=df4.assign(AP_PaO2="")
df4=df4.assign(AP_pH="")
df4=df4.assign(AP_Na="")
df4=df4.assign(AP_K="")
df4=df4.assign(AP_Creatinine="")
df4=df4.assign(AP_WBC="")
df4=df4.assign(AP_GCS="")
df4=df4.assign(AP_HCO3="")
df4=df4.assign(AP_age="")

df4=df4.assign(AP_sum="")

In [ ]:
df4.loc[(df4['BT']>=41) | (df4['BT']<=29.9), 'AP_Tem'] = 4
df4.loc[(39<=df4['BT']) & (df4['BT']<=40.9), 'AP_Tem'] = 3
df4.loc[(30<=df4['BT']) & (df4['BT']<=31.9), 'AP_Tem'] = 3
df4.loc[(32<=df4['BT']) & (df4['BT']<=33.9), 'AP_Tem'] = 2
df4.loc[(38.5<=df4['BT']) & (df4['BT']<= 38.9), 'AP_Tem'] = 1
df4.loc[(34<=df['BT']) & (df4['BT']<=35.9), 'AP_Tem'] = 1
df4.loc[(36<=df4['BT']) & (df['BT']<= 38.4), 'AP_Tem'] = 0

In [ ]:
df4.loc[df4['MBP']>=160 | (df4['MBP']<=49), 'AP_MBP'] = 4
df4.loc[(130<=df4['MBP']) & (df4['MBP']<=159), 'AP_MBP'] = 3
df4.loc[(110<=df4['MBP']) & (df4['MBP']<=129), 'AP_MBP'] = 2
df4.loc[(50<=df4['MBP']) & (df4['MBP']<=69), 'AP_MBP'] = 2
df4.loc[(70<=df4['MBP']) & (df['MBP']<= 109), 'AP_MBP'] = 0

In [ ]:
df4.loc[df4['HR']>=180 | (df4['HR']<=39), 'AP_HR'] = 4
df4.loc[(140<=df4['HR']) & (df4['HR']<=17), 'AP_HR'] = 3
df4.loc[(40<=df4['HR']) & (df4['HR']<=54), 'AP_HR'] = 3
df4.loc[(110<=df4['HR']) & (df4['HR']<=139), 'AP_HR'] = 2
df4.loc[(55<=df4['HR']) & (df4['HR']<=69), 'AP_HR'] = 2
df4.loc[(70<=df4['HR']) & (df['HR']<= 109), 'AP_HR'] = 0

In [ ]:
df4.loc[(df4['RR']>=50) | (df4['RR']<=5), 'AP_RR'] = 4
df4.loc[(35<=df4['RR']) & (df4['RR']<=49), 'AP_RR'] = 3
df4.loc[(6<=df4['RR']) & (df4['RR']<=9), 'AP_RR'] = 2
df4.loc[(25<=df4['RR']) & (df4['RR']<= 34), 'AP_RR'] = 1
df4.loc[(10<=df['RR']) & (df4['RR']<=11), 'AP_RR'] = 1
df4.loc[(12<=df4['RR']) & (df['RR']<= 24), 'AP_RR'] = 0

In [ ]:
df4.loc[(df4['PaO2']>=55), 'AP_PaO2'] = 4
df4.loc[(55<=df4['PaO2']) & (df4['PaO2']<=60), 'AP_PaO2'] = 3
df4.loc[(61<=df4['PaO2']) & (df4['PaO2']<= 70), 'AP_PaO2'] = 1
df4.loc[(70<=df4['PaO2']) , 'AP_PaO2'] = 0

In [ ]:
df4.loc[(df4['pH']>=7.7) | (df4['pH']<=7.15), 'AP_pH'] = 4
df4.loc[(7.6<=df4['pH']) & (df4['pH']<=7.69), 'AP_pH'] = 3
df4.loc[(7.15<=df4['pH']) & (df4['pH']<=7.24), 'AP_pH'] = 3
df4.loc[(7.25<=df4['pH']) & (df4['pH']<=7.32), 'AP_pH'] = 2
df4.loc[(7.5<=df4['pH']) & (df4['pH']<=7.59), 'AP_pH'] = 1
df4.loc[(7.33<=df4['pH']) & (df['pH']<= 7.49), 'AP_pH'] = 0

In [ ]:
df4.loc[(df4['Na']>=180) | (df4['Na']<=110), 'AP_Na'] = 4
df4.loc[(160<=df4['Na']) & (df4['Na']<=179), 'AP_Na'] = 3
df4.loc[(111<=df4['Na']) & (df4['Na']<=119), 'AP_Na'] = 3
df4.loc[(155<=df4['Na']) & (df4['Na']<=159), 'AP_Na'] = 2
df4.loc[(120<=df4['Na']) & (df4['Na']<=129), 'AP_Na'] = 2
df4.loc[(150<=df4['Na']) & (df4['Na']<= 154), 'AP_Na'] = 1
df4.loc[(130<=df4['Na']) & (df['Na']<= 149), 'AP_Na'] = 0

In [ ]:
df4.loc[(df4['K']>=7) | (df4['K']<=2.5), 'AP_K'] = 4
df4.loc[(6<=df4['K']) & (df4['K']<=6.9), 'AP_K'] = 3
df4.loc[(2.5<=df4['K']) & (df4['K']<=2.9), 'AP_K'] = 2
df4.loc[(5.5<=df4['K']) & (df4['K']<=5.9), 'AP_K'] = 2
df4.loc[(3<=df4['K']) & (df4['K']<= 3.4), 'AP_K'] = 1
df4.loc[(3.5<=df4['K']) & (df['K']<= 5.4), 'AP_K'] = 0

In [ ]:
df4.loc[(df4['Creatinine']>=3.5), 'AP_Creatinine'] = 4
df4.loc[(2<=df4['Creatinine']) & (df4['Creatinine']<=3.4), 'AP_Creatinine'] = 3
df4.loc[(1.5<=df4['Creatinine']) & (df4['Creatinine']<=1.9), 'AP_Creatinine'] = 2
df4.loc[(0.6<=df4['Creatinine']), 'AP_Creatinine'] = 2
df4.loc[(0.6<=df4['Creatinine']) & (df['Creatinine']<= 1.4), 'AP_Creatinine'] = 0

In [ ]:
df4.loc[(df4['WBC']>=40) | (df4['WBC']<=1), 'AP_WBC'] = 4
df4.loc[(20<=df4['WBC']) & (df4['WBC']<=39.9), 'AP_WBC'] = 2
df4.loc[(1<=df4['WBC']) & (df4['WBC']<=2.9), 'AP_WBC'] = 2
df4.loc[(15<=df4['WBC']) & (df4['WBC']<= 19.9), 'AP_WBC'] = 1
df4.loc[(3<=df4['WBC']) & (df['WBC']<= 14.9), 'AP_WBC'] = 0

In [ ]:
df4['AP_GCS'] = 15- df4['GCS']

In [ ]:
df4.loc[(df4['HCO3']>=52) | (df4['HCO3']<=15), 'AP_HCO3'] = 4
df4.loc[(41<=df4['HCO3']) & (df4['HCO3']<=51.9), 'AP_HCO3'] = 3
df4.loc[(15<=df4['HCO3']) & (df4['HCO3']<=17.9), 'AP_HCO3'] = 3
df4.loc[(18<=df4['HCO3']) & (df4['HCO3']<=21.9), 'AP_HCO3'] = 2
df4.loc[(32<=df4['HCO3']) & (df4['HCO3']<= 40.9), 'AP_HCO3'] = 1
df4.loc[(22<=df4['HCO3']) & (df['HCO3']<= 31.9), 'AP_HCO3'] = 0

In [ ]:
df4.loc[(df4['age']<=44), 'AP_age'] = 0
df4.loc[(45<=df4['age']) & (df4['age']<=54), 'AP_age'] = 2
df4.loc[(55<=df4['age']) & (df4['age']<=64), 'AP_age'] = 3
df4.loc[(65<=df4['age']) & (df4['age']<= 74), 'AP_age'] = 5
df4.loc[(df4['age']<=75), 'AP_age'] = 6

In [ ]:
df4

,pid,age,Gender,ICUType,ALP,ALT,AST,Albumin,BUN,Bilirubin,...,AP_PaO2,AP_pH,AP_Na,AP_K,AP_Creatinine,AP_WBC,AP_GCS,AP_HCO3,AP_age,AP_sum
0,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,0,,,,2,,2.0,,6,
1,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,0,,,,2,,2.0,,6,
2,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,0,,,,2,,2.0,,6,
3,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,0,,,,2,,2.0,,6,
4,142219,52.0,0.0,3.0,82.0,43.0,63.0,2.9,20.0,0.9,...,0,,,,2,,2.0,,6,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,0,,,,2,,7.0,,6,
242609,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,0,,,,2,,7.0,,6,
242610,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,0,,,,2,,7.0,,6,
242611,138581,53.0,1.0,4.0,82.0,43.0,63.0,2.9,18.0,0.9,...,0,,,,2,,7.0,,6,


In [ ]:
# 필요 컬럼들만 가져오기 'pid' + 'AP_~' + 'AP_sum'
df5 = df4[['pid','AP_Tem','AP_MBP','AP_HR','AP_RR','AP_PaO2','AP_pH','AP_Na','AP_K','AP_Creatinine','AP_WBC','AP_GCS','AP_HCO3','AP_age','AP_sum']]
df5


,pid,AP_Tem,AP_MBP,AP_HR,AP_RR,AP_PaO2,AP_pH,AP_Na,AP_K,AP_Creatinine,AP_WBC,AP_GCS,AP_HCO3,AP_age,AP_sum
0,142219,0,0,0,0,0,,,,2,,2.0,,6,
1,142219,0,0,0,0,0,,,,2,,2.0,,6,
2,142219,0,4,0,0,0,,,,2,,2.0,,6,
3,142219,0,3,0,0,0,,,,2,,2.0,,6,
4,142219,0,3,0,0,0,,,,2,,2.0,,6,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,0,0,0,0,0,,,,2,,7.0,,6,
242609,138581,0,0,0,0,0,,,,2,,7.0,,6,
242610,138581,0,0,0,0,0,,,,2,,7.0,,6,
242611,138581,0,0,0,0,0,,,,2,,7.0,,6,


In [ ]:
df5['AP_pH'].value_counts()

     192459
2     24014
0     14679
3      5147
1      4955
4      1359
Name: AP_pH, dtype: int64

In [ ]:
# 새로 만든 변수에 결측치 0점 처리해주기
df5.fillna(0)

,pid,AP_Tem,AP_MBP,AP_HR,AP_RR,AP_PaO2,AP_pH,AP_Na,AP_K,AP_Creatinine,AP_WBC,AP_GCS,AP_HCO3,AP_age,AP_sum
0,142219,0,0,0,0,0,,,,2,,2.0,,6,
1,142219,0,0,0,0,0,,,,2,,2.0,,6,
2,142219,0,4,0,0,0,,,,2,,2.0,,6,
3,142219,0,3,0,0,0,,,,2,,2.0,,6,
4,142219,0,3,0,0,0,,,,2,,2.0,,6,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,0,0,0,0,0,,,,2,,7.0,,6,
242609,138581,0,0,0,0,0,,,,2,,7.0,,6,
242610,138581,0,0,0,0,0,,,,2,,7.0,,6,
242611,138581,0,0,0,0,0,,,,2,,7.0,,6,


In [ ]:
df5.isnull().sum()

pid              0
AP_Tem           0
AP_MBP           0
AP_HR            0
AP_RR            0
AP_PaO2          0
AP_pH            0
AP_Na            0
AP_K             0
AP_Creatinine    0
AP_WBC           0
AP_GCS           0
AP_HCO3          0
AP_age           0
AP_sum           0
dtype: int64

In [ ]:
df5['AP_pH'].value_counts()

     192459
2     24014
0     14679
3      5147
1      4955
4      1359
Name: AP_pH, dtype: int64

In [ ]:
df5['AP_WBC'].unique()

array(['', 0, 4, 2, 1], dtype=object)

In [ ]:
df5['AP_Tem'] = df5['AP_Tem'].apply(lambda x:0 if x=='' else x)
df5['AP_MBP'] = df5['AP_MBP'].apply(lambda x:0 if x=='' else x)
df5['AP_HR'] = df5['AP_HR'].apply(lambda x:0 if x=='' else x)
df5['AP_RR'] = df5['AP_RR'].apply(lambda x:0 if x=='' else x)
df5['AP_PaO2'] = df5['AP_PaO2'].apply(lambda x:0 if x=='' else x)
df5['AP_pH'] = df5['AP_pH'].apply(lambda x:0 if x=='' else x)
df5['AP_Na'] = df5['AP_Na'].apply(lambda x:0 if x=='' else x)
df5['AP_K'] = df5['AP_K'].apply(lambda x:0 if x=='' else x)
df5['AP_Creatinine'] = df5['AP_Creatinine'].apply(lambda x:0 if x=='' else x)
df5['AP_WBC'] = df5['AP_WBC'].apply(lambda x:0 if x=='' else x)
df5['AP_GCS'] = df5['AP_GCS'].apply(lambda x:0 if x=='' else x)
df5['AP_HCO3'] = df5['AP_HCO3'].apply(lambda x:0 if x=='' else x)
df5['AP_age'] = df5['AP_age'].apply(lambda x:0 if x=='' else x)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
df5['AP_sum'] = df5['AP_Tem'] + df5['AP_MBP'] + df5['AP_HR'] + df5['AP_RR'] + df5['AP_PaO2']+ df5['AP_pH'] + df5['AP_Na']+ df5['AP_K'] + df5['AP_Creatinine']+ df5['AP_WBC'] + df5['AP_GCS']+ df5['AP_HCO3'] + df5['AP_age']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df5

,pid,AP_Tem,AP_MBP,AP_HR,AP_RR,AP_PaO2,AP_pH,AP_Na,AP_K,AP_Creatinine,AP_WBC,AP_GCS,AP_HCO3,AP_age,AP_sum
0,142219,0,0,0,0,0,0,0,0,2,0,2.0,0,6,10.0
1,142219,0,0,0,0,0,0,0,0,2,0,2.0,0,6,10.0
2,142219,0,4,0,0,0,0,0,0,2,0,2.0,0,6,14.0
3,142219,0,3,0,0,0,0,0,0,2,0,2.0,0,6,13.0
4,142219,0,3,0,0,0,0,0,0,2,0,2.0,0,6,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242608,138581,0,0,0,0,0,0,0,0,2,0,7.0,0,6,15.0
242609,138581,0,0,0,0,0,0,0,0,2,0,7.0,0,6,15.0
242610,138581,0,0,0,0,0,0,0,0,2,0,7.0,0,6,15.0
242611,138581,0,0,0,0,0,0,0,0,2,0,7.0,0,6,15.0


In [ ]:
df.to_csv('APACHE2-.csv')

In [ ]:
df5.to_csv('df5.csv')

In [ ]:
df_survival__.to_csv('df_survival__.csv')